In [1]:
import numpy as np
import pandas as pd
import fuzzymatcher
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
df = pd.read_csv('aka.csv', encoding = "ISO-8859-1", dtype='unicode')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 12 columns):
id                 1500 non-null object
movie_id           1500 non-null object
title              1500 non-null object
imdb_index         0 non-null object
kind_id            1500 non-null object
production_year    1423 non-null object
phonetic_code      1483 non-null object
episode_of_id      77 non-null object
season_nr          69 non-null object
episode_nr         69 non-null object
note               1248 non-null object
md5sum             1500 non-null object
dtypes: object(12)
memory usage: 140.7+ KB


In [3]:
df_right = df[df.kind_id != "7"]

df_right.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1423 entries, 0 to 1499
Data columns (total 12 columns):
id                 1423 non-null object
movie_id           1423 non-null object
title              1423 non-null object
imdb_index         0 non-null object
kind_id            1423 non-null object
production_year    1423 non-null object
phonetic_code      1406 non-null object
episode_of_id      0 non-null object
season_nr          0 non-null object
episode_nr         0 non-null object
note               1225 non-null object
md5sum             1423 non-null object
dtypes: object(12)
memory usage: 144.5+ KB


In [4]:
df_right.describe()

,id,movie_id,title,imdb_index,kind_id,production_year,phonetic_code,episode_of_id,season_nr,episode_nr,note,md5sum
count,1423,1423,1423,0.0,1423,1423,1406,0.0,0.0,0.0,1225,1423
unique,1423,966,1372,0.0,2,64,929,0.0,0.0,0.0,184,1415
top,1860,946272,Der KopfgeldjÃ¤ger,NaN,2,1959,H4562,NaN,NaN,NaN,(USA) (anthology series),0162bb276bdcd2a2f4d69677610dba28
freq,1,13,3,NaN,1052,151,189,NaN,NaN,NaN,188,2


In [5]:
df_left = pd.read_csv('enc.csv', encoding = "ISO-8859-1")
df_left.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732 entries, 0 to 1731
Data columns (total 12 columns):
Number                 1732 non-null int64
Program Title          1732 non-null object
Program Type           1731 non-null object
Program Genre          1731 non-null object
Network                1704 non-null object
First Air Month        131 non-null float64
First Air Day          123 non-null float64
First Air Year         1732 non-null int64
Last Air Month         2 non-null float64
Last Air Day           2 non-null float64
Last Air Year          895 non-null float64
Program Description    1732 non-null object
dtypes: float64(5), int64(2), object(5)
memory usage: 162.5+ KB


In [6]:
df_left ["Program Genre"].value_counts()

 Variety            389
 Anthology          275
 Comedy             226
 Game               169
 Adventure          144
 Crime Drama        124
 Drama               88
 Western             86
 Children            72
 Serial              33
 Cartoon             24
 Interview           19
 Women               18
 Human Interest      12
 Documentary         11
 Travel               7
 Mystery              6
 Discussion           6
 Comedy-Drama         5
 Science Fiction      4
 Music                3
 Contest              2
 Advice               2
 Testimonial          1
 Tribute              1
 Cooking              1
 Health               1
 Musical Drama        1
 Information          1
Name: Program Genre, dtype: int64

In [29]:
df_right = df_right.astype(str).apply(lambda x: x.str.lower())
df_left = df_left.astype(str).apply(lambda x: x.str.lower())

In [30]:
#Dropped columns by cutting out in excel. See diff between title4 & title5
#df_right.drop(df_right.columns[[2,5,7,8,9,20]], axis=1, inplace=True)

In [31]:
df_right.head()

,id,movie_id,title,imdb_index,kind_id,production_year,phonetic_code,episode_of_id,season_nr,episode_nr,note,md5sum
0,41,2514,kurze pause...,nan,2,1956,k6212,nan,nan,nan,(west germany) (alternative title),5956ba67e6567ef5b1c94ad51bf96947
1,233,11762,1:0 fã¼r ... dã¼sseldorf,nan,2,1954,f6324,nan,nan,nan,nan,598c71ea24df9aecb1d2ca9f143f1f1e
2,234,11762,1:0 fã¼r... dich,nan,2,1954,f632,nan,nan,nan,nan,c5cbc0222e09722578e79ae11990da06
3,330,16535,twenty-one beacon street,nan,2,1959,t5351,nan,nan,nan,nan,6ed2faf7be692a467b3d26238458683f
4,559,34862,7 wã¼nsche die sie haben - die wir erfã¼llen,nan,2,1956,w5232,nan,nan,nan,(west germany) (alternative title),f7560017b0af07dcf4b5550ebe36a82b


In [32]:
df_right.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1423 entries, 0 to 1499
Data columns (total 12 columns):
id                 1423 non-null object
movie_id           1423 non-null object
title              1423 non-null object
imdb_index         1423 non-null object
kind_id            1423 non-null object
production_year    1423 non-null object
phonetic_code      1423 non-null object
episode_of_id      1423 non-null object
season_nr          1423 non-null object
episode_nr         1423 non-null object
note               1423 non-null object
md5sum             1423 non-null object
dtypes: object(12)
memory usage: 144.5+ KB


In [33]:
df_left.drop(df_left.head().columns[[5,6,8,9]], axis=1, inplace=True)

In [34]:
df_left.head()

,Number,Program Title,Program Type,Program Genre,Network,First Air Year,Last Air Year,Program Description
0,5137,mary kay and johnny.,series,comedy,"dumont, nbc, cbs",1947,1950.0,the marital misadventures of mary kay and joh...
1,7531,the small fry club.,series,children,dumont,1947,1951.0,"bob emery hosts a program of stories, game co..."
2,1738,crawford mystery theater.,series,game,dumont,1947,1952.0,the alternate title for public prosecutor (wh...
3,6636,public prosecutor.,series,mystery,"syn., dumont",1947,1952.0,the syndicated version of the series (1947-48...
4,4323,juvenile jury.,series,children,"nbc, cbs",1947,1955.0,the program features a panel of five children...


In [35]:
df_right["title"].head()

0                                  kurze pause...
1                        1:0 fã¼r ... dã¼sseldorf
2                                1:0 fã¼r... dich
3                        twenty-one beacon street
4    7 wã¼nsche die sie haben - die wir erfã¼llen
Name: title, dtype: object

In [36]:
df_left["Program Title"] = df_left["Program Title"].str.replace('[.]','')

In [37]:
df_left["Program Title"]

0                   mary kay and johnny 
1                    the small fry club 
2              crawford mystery theater 
3                     public prosecutor 
4                         juvenile jury 
5          the kraft television theater 
6                 kukla, fran and ollie 
7                           howdy doody 
8                    midwestern hayride 
9                        actor's studio 
10                  the bob howard show 
11               the broadway playhouse 
12                    cartoon teletales 
13         the chesterfield supper club 
14           the chevrolet tele-theater 
15             the morey amsterdam show 
16                  the ted steele show 
17                           tv shopper 
18                         village barn 
19                   the alan dale show 
20                               club 7 
21                    ford theater hour 
22                           lticky pup 
23                          okay mother 
24              

In [38]:
(process.extract(x, df_right, scorer= fuzz.ratio) for x in df_left)

<generator object <genexpr> at 0x10a5780f8>

In [39]:
#This works but needs tweaking

names_array=[]
ratio_array=[]

def match_names(wrong_names,correct_names):
    for row in wrong_names:
        x=process.extractOne(row, correct_names)
        names_array.append(x[0])
        ratio_array.append(x[1])
    return names_array,ratio_array
 
 
#imdb names dataset

wrong_names=df_left["Program Title"].dropna().values
 
#enc names dataset
correct_names=df_right["title"].dropna().values
 
name_match,ratio_match=match_names(wrong_names,correct_names)
 
df_left['correct_name']=pd.Series(name_match)
df_left['name_ratio']=pd.Series(ratio_match)
 
 
print(df_left[['Program Title','correct_name','name_ratio']].head(10))

                    Program Title                         correct_name  \
0            mary kay and johnny   captain video and his video rangers   
1             the small fry club          peter jennings with the news   
2       crawford mystery theater                       mystery theater   
3              public prosecutor                     public prosecutor   
4                  juvenile jury                                  fury   
5   the kraft television theater                    the al morgan show   
6          kukla, fran and ollie   captain video and his video rangers   
7                    howdy doody                           howdy doody   
8             midwestern hayride                 wilder westen arizona   
9                 actor's studio    korla pandit's adventures in music   

   name_ratio  
0          86  
1          86  
2          90  
3         100  
4          68  
5          86  
6          86  
7         100  
8          62  
9          86  


In [21]:
#df_right.to_csv("testing2.csv")

In [40]:
df_merge = pd.merge(df_left, df_right, left_on= "correct_name", right_on = "title")

In [41]:
print(df_merge.head(10))

  Number                          Program Title  Program Type  \
0   5137                   mary kay and johnny         series   
1   4498                 kukla, fran and ollie         series   
2    340                   and everything nice         series   
3   1236   captain video and his video rangers         series   
4   5673                       mystery and mrs   test program   
5   4400                      kids and company         series   
6   1033                       bride and groom         series   
7   7281                    sense and nonsense         series   
8   5404                      mr and mrs north         series   
9   1891                      dave and charley         series   

      Program Genre            Network First Air Year Last Air Year  \
0            comedy   dumont, nbc, cbs           1947        1950.0   
1          children           nbc, abc           1947        1957.0   
2             women             dumont           1949        1950.0   


In [42]:
df_merge.to_csv("merged-aka.csv")
###########

In [155]:
#DOES NOT WORK
#Columns to match on from df_left

df_left.dropna().values
df_right.dropna().values

left_on = ["Program Title", "First Air Year"]

# Columns to match on from df_right
right_on = ["title", "production_year"]

fuzzymatcher.link_table(df_left, df_right, left_on, right_on)

ZeroDivisionError: float division by zero

In [158]:
#DOES NOT WORK
fuzzymatcher.fuzzy_left_join(df_left, df_right, left_on = "Program Title", right_on = "title")

ZeroDivisionError: float division by zero